<a href="https://colab.research.google.com/github/Tanmay-jam/Conversational-chatbot-with-finutuned-BART/blob/main/Loss%20Evaluation%20and%20chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Task 1 Summary generation using LLama for ground truth of 1941 training data points

Previously we have generated summaries LLama 3 which is considered as ground truth.
In the second task, we will evaluate a pretrained model for summarization task and try to fine tune it for summarization and evaluate finetuned model to see if model better adapts to summarization.

This notebook consists of
<ul><b>Part 1</b>
<li>Fine tuning model for summarization task</li>
<li>Generating summaries using pretrained and finetuned model</li>
<li>Loss evaluation of generated summaries on training and validation data</li>
</ul>
<ul><b>Part 2</b>
<li>Chatbot with Finetuned model for summarization of conversation and another DioloGPT for next chat prediction</li></ul>

# Part 1: Finetuning and Evaluation

In [ ]:
# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

In [ ]:
import pandas as pd
import torch
from transformers import BartForConditionalGeneration, BartTokenizer, BlenderbotTokenizer, BlenderbotForConditionalGeneration

In [ ]:
df_train = pd.read_csv('/content/total_train_data.csv')
df_train.head()

,Conversation,Summaries
0,"say, jim, how about going for a few beers afte...","Jim suggests going for beers after dinner, but..."
1,can you do push-ups? of course i can. it's a p...,Person A claims to be able to do 30 push-ups p...
2,"can you study with the radio on? no, i listen ...","Person A suggests studying with the radio on, ..."
3,are you all right? i will be all right soon. i...,"Person A asks if everything is okay, reassures..."
4,"hey john, nice skates. are they new? yeah, i j...",John and another person discuss their new ice ...


In [ ]:
data = df_train.copy()
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1941 entries, 0 to 1940
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Conversation  1941 non-null   object
 1   Summaries     1941 non-null   object
dtypes: object(2)
memory usage: 30.5+ KB


In df_train dataframe, column <b>Conversation</b> is the dialogue from DailyDailogue dataset and column <b>Summaries</b> is summaries of conversation generated using LLama 3 (treated as our ground thruth summaries)

## Fine tuning BART Large Xsum on our data

The model is finetuned in different notebook here:
https://colab.research.google.com/drive/1zjammUQRpa-VEgF6HdEekvtdFdVIVUtM?usp=sharing

## Generate summaries using BART-Large-xsum model on training data

### Generating summaries using pretrained BART-Large-xsum

In [ ]:
def summarize_conversation(convo):

    """Summarize the entire conversation history."""
    #dialogue = " ".join(history[-10:])  # Use the last 10 exchanges for summarization
    inputs = bart_tokenizer(convo , return_tensors="pt", max_length=1024, truncation=True).to(device)
    summary_ids = bart_model.generate(
        inputs["input_ids"],
        num_beams=4,
        min_length = 10,
        max_length=100,
        length_penalty=2.0)
    summary = bart_tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    print(f"Summarized Dialogue: {summary}")
    print("-" * 50)
    return summary

In [ ]:
# Load tokenizer and base model (BART-Large-xsum)
model_name = "facebook/bart-large-xsum"
bart_tokenizer = BartTokenizer.from_pretrained(model_name)
bart_model = BartForConditionalGeneration.from_pretrained(model_name).to(device)


from tqdm import tqdm
tqdm.pandas()

# Generate BART summaries and add them as a new column
data['bartFinetuned_summary'] = data['Conversation'].progress_apply(summarize_conversation)

  0%|          | 2/1941 [00:02<38:08,  1.18s/it]

Summarized Dialogue: Jim suggests going out for a few beers after dinner, which is tempting but not good for fitness or relaxation, as it will just make them fat and make them act silly. Instead, they suggest a walk to the gym where they can play singsong and meet some friends to play pingpong and possibly ask them to dance with them, which would be fun and exercise.
--------------------------------------------------


  0%|          | 3/1941 [00:02<29:12,  1.11it/s]

Summarized Dialogue: Person A proves they can do 30 push-ups in 30 seconds, proving it's possible to do it with exercise.
--------------------------------------------------


  0%|          | 4/1941 [00:03<27:31,  1.17it/s]

...

Summarized Dialogue: The speaker is pregnant and the doctor told them to start listening to Bach music to improve their baby's brain development. They bought books on having kids and were told they should play classical music to make their baby smarter.
--------------------------------------------------


100%|██████████| 1941/1941 [34:40<00:00,  1.39it/s]

Summarized Dialogue: Pluto is no longer considered a planet, due to a change in the solar system's definition of size.
--------------------------------------------------


100%|██████████| 1941/1941 [34:40<00:00,  1.07s/it]

Summarized Dialogue: Emily recently finished school, and is relieved to be done with school. She plans to go to university in September, but her admission depends on passing her a level subjects.
--------------------------------------------------


In [ ]:
data.to_csv('bartXsum_summaries.csv', index=False)

### Upload model files

In [ ]:
from google.colab import files
import zipfile
import os

# Upload the zip file
uploaded = files.upload()  # This will prompt you to upload the zip file manually

# Unzip the uploaded file
zip_filename = next(iter(uploaded))  # Gets the uploaded file name

# Create a directory (optional, if you want to extract to a specific folder)
extract_dir = './bartXsum_finetuned'
os.makedirs(extract_dir, exist_ok=True)

# Unzip
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Files extracted to: {extract_dir}")


Saving bartXsum_finetuned.zip to bartXsum_finetuned.zip
Files extracted to: ./bartXsum_finetuned


### Generating summaries using Finetuned BART-Large-xsum

In [ ]:
# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)


# Load the fine-tuned BART model for summarization
bart_model_path = "./bartXsum_finetuned"  # Path to your fine-tuned model
bart_model = BartForConditionalGeneration.from_pretrained(bart_model_path).to(device)
bart_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-xsum")


from tqdm import tqdm
tqdm.pandas()

# Generate BART summaries and add them as a new column
data['bartFinetuned_summary'] = data['Conversation'].progress_apply(summarize_conversation)

  0%|          | 2/1941 [00:02<38:08,  1.18s/it]

Summarized Dialogue: Jim suggests going out for a few beers after dinner, which is tempting but not good for fitness or relaxation, as it will just make them fat and make them act silly. Instead, they suggest a walk to the gym where they can play singsong and meet some friends to play pingpong and possibly ask them to dance with them, which would be fun and exercise.
--------------------------------------------------


  0%|          | 3/1941 [00:02<29:12,  1.11it/s]

Summarized Dialogue: Person A proves they can do 30 push-ups in 30 seconds, proving it's possible to do it with exercise.
--------------------------------------------------


  0%|          | 4/1941 [00:03<27:31,  1.17it/s]

...

Summarized Dialogue: The speaker is pregnant and the doctor told them to start listening to Bach music to improve their baby's brain development. They bought books on having kids and were told they should play classical music to make their baby smarter.
--------------------------------------------------


100%|██████████| 1941/1941 [34:40<00:00,  1.39it/s]

Summarized Dialogue: Pluto is no longer considered a planet, due to a change in the solar system's definition of size.
--------------------------------------------------


100%|██████████| 1941/1941 [34:40<00:00,  1.07s/it]

Summarized Dialogue: Emily recently finished school, and is relieved to be done with school. She plans to go to university in September, but her admission depends on passing her a level subjects.
--------------------------------------------------


In [ ]:
data.to_csv('bartXsumFinetuned_summaries.csv', index=False)

## Loss Evaluation

In [ ]:
!pip install evaluate rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=b853963d72180ec0bd0dad740354d5e07ec8616d79ac67d6d77946c11b052030
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge_score


In [ ]:
import evaluate

In [ ]:
data = pd.read_csv('/content/bartXsum_summaries.csv')
data_finetuned = pd.read_csv('/content/bartXsumFinetuned_summaries.csv')

In [ ]:
data.info(), data_finetuned.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1941 entries, 0 to 1940
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   Conversation  1941 non-null   object
 1   Summaries     1941 non-null   object
 2   bart_summary  1941 non-null   object
dtypes: object(3)
memory usage: 45.6+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1941 entries, 0 to 1940
Data columns (total 3 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Conversation           1941 non-null   object
 1   Summaries              1941 non-null   object
 2   bartFinetuned_summary  1941 non-null   object
dtypes: object(3)
memory usage: 45.6+ KB


(None, None)

### Evaluation on training data

In [ ]:
df1 = pd.read_csv('/content/bartXsum_summaries.csv')
df2 = pd.read_csv('/content/bartXsumFinetuned_summaries.csv')

# Drop the common columns from df2 to avoid duplication
df2_trimmed = df2.drop(columns=['Conversation', 'Summaries'])

# Concatenate column-wise (side-by-side)
merged = pd.concat([df1, df2_trimmed], axis=1)

In [ ]:
merged.head()

,Conversation,Summaries,bart_summary,bartFinetuned_summary
0,"say, jim, how about going for a few beers afte...","Jim suggests going for beers after dinner, but...","Jim and i are sitting at home in our pyjamas, ...",Jim suggests going out for a few beers after d...
1,can you do push-ups? of course i can. it's a p...,Person A claims to be able to do 30 push-ups p...,Do you think you can do 30 push-ups in a minute?,Person A proves they can do 30 push-ups in 30 ...
2,"can you study with the radio on? no, i listen ...","Person A suggests studying with the radio on, ...",Do you listen to the radio or the record player?,The conversation discusses the difference betw...
3,are you all right? i will be all right soon. i...,"Person A asks if everything is okay, reassures...",This is the moment a man and a woman fell from...,Person A was terrified when they fell from a w...
4,"hey john, nice skates. are they new? yeah, i j...",John and another person discuss their new ice ...,"A few weeks ago, i was talking to a guy named ...",John recently got new skates for playing ice h...


#### Rouge score of pretrained and fine-tuned model

In [ ]:
# Load the ROUGE metric
rouge = evaluate.load("rouge")

# Extract the reference and prediction lists
references = merged['Summaries'].tolist()      # Replace with your actual reference summary generated using llama for ground truth


predictions = merged['bart_summary'].tolist()
predictions_finetuned = merged['bartFinetuned_summary'].tolist()
# Compute ROUGE scores
result = rouge.compute(predictions=predictions, references=references, use_stemmer=True)
result_finetuned = rouge.compute(predictions=predictions_finetuned, references=references, use_stemmer=True)
print("ROUGE scores for Bart large Xsum:", result)
print('-'*50)
print("ROUGE scores for Finetuned Bart large Xsum:", result_finetuned)

ROUGE scores for Bart large Xsum: {'rouge1': np.float64(0.176974756479149), 'rouge2': np.float64(0.03323407642518825), 'rougeL': np.float64(0.13342882300181538), 'rougeLsum': np.float64(0.1334425898820679)}
--------------------------------------------------
ROUGE scores for Finetuned Bart large Xsum: {'rouge1': np.float64(0.41970279545712597), 'rouge2': np.float64(0.174778470134078), 'rougeL': np.float64(0.32363005892219016), 'rougeLsum': np.float64(0.3234860231948047)}


#### METEOR score for pretrained and fine-tuned model

In [ ]:
import evaluate

meteor = evaluate.load('meteor')

# Extract the reference and prediction lists
references = merged['Summaries'].tolist()      # Replace with your actual reference summary generated using llama for ground truth


predictions = merged['bart_summary'].tolist()
predictions_finetuned = merged['bartFinetuned_summary'].tolist()

# Compute the METEOR score
meteor_result = meteor.compute(predictions=predictions, references=references)
meteor_result_finetuned = meteor.compute(predictions=predictions_finetuned, references=references)
print("METEOR score for Bart large Xsum::", meteor_result)
print('-'*50)
print("METEOR score for Finetuned Bart large Xsum:", meteor_result_finetuned)


[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


METEOR score for Bart large Xsum:: {'meteor': np.float64(0.12179553488686014)}
--------------------------------------------------
METEOR score for Finetuned Bart large Xsum: {'meteor': np.float64(0.3778776732526368)}


#### Bert score for pretrained and fine-tuned model

In [ ]:
!pip install bert_score

ERROR: Operation cancelled by user
Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/base_command.py", line 179, in exc_logging_wrapper
    status = run_func(*args)
             ^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 67, in wrapper
    return func(self, options, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/commands/install.py", line 362, in run
    resolver = self.make_resolver(
               ^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/cli/req_command.py", line 177, in make_resolver
    return pip._internal.resolution.resolvelib.resolver.Resolver(
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 58, in __init__
    self.factory = Factory(
                  

In [ ]:
# Load the BERTScore metric
bertscore = evaluate.load("bertscore")

In [ ]:
# Extract the reference and prediction lists
references = merged['Summaries'].tolist()      # Replace with your actual reference summary generated using llama for ground truth


predictions = merged['bart_summary'].tolist()
predictions_finetuned = merged['bartFinetuned_summary'].tolist()


# Compute BERTScore; you can set model_type, e.g., 'bert-base-uncased'
bertscore_result = bertscore.compute(predictions=predictions,
                                       references=references,
                                       model_type="bert-base-uncased")
bertscore_result_finetuned = bertscore.compute(predictions=predictions_finetuned,
                                       references=references,
                                       model_type="bert-base-uncased")
print("BERTScore for Bart large Xsum::", bertscore_result)
print('-'*50)
print("BERTScore for Finetuned Bart large Xsum:", bertscore_result_finetuned)

BERTScore for Bart large Xsum:: {'precision': [0.4519609212875366, 0.6494698524475098, 0.5597119927406311, 0.41863900423049927, 0.48613089323043823, 0.6037143468856812, 0.4728938043117523, 0.5762585401535034, 0.6607747673988342, 0.41431304812431335, 0.43135660886764526, 0.40456998348236084, 0.39652982354164124, 0.403046190738678, 0.43574491143226624, 0.5103963613510132, 0.48346036672592163, 0.5008592009544373, 0.32880452275276184, 0.4490204453468323, 0.5721136331558228, 0.42250746488571167, 0.3985029458999634, 0.4000040292739868, 0.507813036441803, 0.536868691444397, 0.474181592464447, 0.5447728633880615, 0.43377602100372314, 0.36722519993782043, 0.37287211418151855, 0.4237931966781616, 0.4407747685909271, 0.4436757564544678, 0.4725584089756012, 0.4016875922679901, 0.4640248417854309, 0.4666955769062042, 0.37923258543014526, 0.44052448868751526, 0.5053912401199341, 0.5230792164802551, 0.41949164867401123, 0.4961758255958557, 0.40711483359336853, 0.3532298803329468, 0.4217912256717682, 

In [ ]:
# Print overall scores
print('BERTScore analysis for BART Large Xsum on training data')
print("Average Precision:", sum(bertscore_result['precision']) / len(bertscore_result['precision']))
print("Average Recall:", sum(bertscore_result['recall']) / len(bertscore_result['recall']))
print("Average F1:", sum(bertscore_result['f1']) / len(bertscore_result['f1']))

print('-'*50)
print('BERTScore analysis for Finetuned BART Large Xsum on training data')
print("Average Precision:", sum(bertscore_result_finetuned['precision']) / len(bertscore_result_finetuned['precision']))
print("Average Recall:", sum(bertscore_result_finetuned['recall']) / len(bertscore_result_finetuned['recall']))
print("Average F1:", sum(bertscore_result_finetuned['f1']) / len(bertscore_result_finetuned['f1']))

BERTScore analysis for BART Large Xsum on training data
Average Precision: 0.45347471734041533
Average Recall: 0.4229659147403217
Average F1: 0.435878726439277
--------------------------------------------------
BERTScore analysis for Finetuned BART Large Xsum on training data
Average Precision: 0.6335422467595827
Average Recall: 0.6556286589501139
Average F1: 0.6424452613795436


### Validation data evaluation

In [ ]:
import pandas as pd

df_val = pd.read_csv('/content/val.csv')
df_val.head()

,Conversation,Summaries
0,"hey man, you wanna buy some weed? some what? w...","A person offers various drugs to someone else,..."
1,the taxi drivers are on strike again. what for...,"Taxi drivers are striking, demanding the gover..."
2,we've managed to reduce our energy consumption...,Company achieved 15% reduction in energy consu...
3,"believe it or not, tea is the most popular bev...","People from around the world enjoy tea, and Ch..."
4,what are your personal weaknesses? i m afraid...,The person acknowledges their weakness in publ...


In [ ]:
data = df_val.copy()

##### Generating summaries using pretrained BART Large xsum

In [ ]:
# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# Load tokenizer and base model (BART-Large)
model_name = "facebook/bart-large-xsum"
bart_tokenizer = BartTokenizer.from_pretrained(model_name)
bart_model = BartForConditionalGeneration.from_pretrained(model_name).to(device)

In [ ]:
from tqdm import tqdm
tqdm.pandas()

# Generate BART summaries and add them as a new column
data['bart_summary'] = data['Conversation'].progress_apply(summarize_conversation)

  0%|          | 2/400 [00:00<01:03,  6.25it/s]

Summarized Dialogue: A video has been posted online of a man being arrested in New York City for selling drugs.
--------------------------------------------------


  1%|          | 3/400 [00:00<01:16,  5.21it/s]

Summarized Dialogue: The taxi drivers' strike in France has gone on for a second day.
--------------------------------------------------


  1%|          | 4/400 [00:01<02:05,  3.15it/s]

...

Summarized Dialogue: A little girl in her early 20s has been asked to try on a winter hat at a shop in the city.
--------------------------------------------------


100%|██████████| 400/400 [02:23<00:00,  3.23it/s]

Summarized Dialogue: What do you need to know to get your driver's licence?
--------------------------------------------------


100%|██████████| 400/400 [02:24<00:00,  2.77it/s]

Summarized Dialogue: A young woman in her early 20s, who has just arrived in the country from the United States, is asked to change a vase at a shop in Paris.
--------------------------------------------------


##### Generating summaries on validation data using pretrained BART Large xsum

Run below to import fine tuned model files (execute if needed)

In [ ]:
from google.colab import files
import zipfile
import os

# Upload the zip file
uploaded = files.upload()  # This will prompt you to upload the zip file manually

# Unzip the uploaded file
zip_filename = next(iter(uploaded))  # Gets the uploaded file name

# Create a directory (optional, if you want to extract to a specific folder)
extract_dir = './bartXsum_finetuned'
os.makedirs(extract_dir, exist_ok=True)

# Unzip
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Files extracted to: {extract_dir}")


Saving bartXsum_finetuned.zip to bartXsum_finetuned.zip
Files extracted to: ./bartXsum_finetuned


In [ ]:
# Check if GPU is available
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)


# Load the fine-tuned BART model for summarization
bart_model_path = "./bartXsum_finetuned"  # Path to your fine-tuned model
bart_model = BartForConditionalGeneration.from_pretrained(bart_model_path).to(device)
bart_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-xsum")

In [ ]:
from tqdm import tqdm
tqdm.pandas()

# Generate BART summaries and add them as a new column
data['bartFinetuned_summary'] = data['Conversation'].progress_apply(summarize_conversation)

  0%|          | 2/400 [00:01<04:36,  1.44it/s]

Summarized Dialogue: Person A wants to buy weed, marijuana, cannabis, or Mary Jane, but Person B declines. Person A offers to give them a few lines if they prefer, and offers to buy one ounce for free. Person B offers to try out other drugs, including Mary Jane and acid, and suggests they try out some of their connections' products.
--------------------------------------------------


  1%|          | 3/400 [00:01<04:11,  1.58it/s]

Summarized Dialogue: The taxi drivers are on strike again. They want the government to reduce the price of gasoline, which is a hot potato.
--------------------------------------------------


  1%|          | 4/400 [00:03<05:23,  1.22it/s]

...

Summarized Dialogue: The customer finds a red winter hat on sale at a clearance sale and tries it on, finding it to be attractive. The salesperson suggests they try it on in front of a mirror to see if it suits them.
--------------------------------------------------


100%|██████████| 400/400 [06:45<00:00,  1.30it/s]

Summarized Dialogue: Person A asks a question about driving hours needed to get a driver's license. They need six hours of driving experience, which can be obtained in three days.
--------------------------------------------------


100%|██████████| 400/400 [06:46<00:00,  1.02s/it]

Summarized Dialogue: The speaker apologises for their mistake and asks for an excuse, explaining that they found a crack in a vase they bought an hour ago and found it on the bottom when they returned to their hotel. They offer to change the vase for them, and the speaker accepts.
--------------------------------------------------


In [ ]:
data.to_csv('bartXsum_val_summaries.csv', index=False)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Conversation           400 non-null    object
 1   Summaries              400 non-null    object
 2   bart_summary           400 non-null    object
 3   bartFinetuned_summary  400 non-null    object
dtypes: object(4)
memory usage: 12.6+ KB


Now dataframe <b>data</b> contains
<ul>
<li>Conversation - dialogues as data point from dataset</li>
<li>Summaries - Summaries generated by LLama 3 (Ground truth)</li>
<li>bart_summary - Summaries generated by pretrained BART-Large-Xsum model</li>
<li>bartFinetuned_summary = Summaries generated by fintuned BART-Large-Xsum model</li>
</ul>

##### Rouge score of pretrained and fine-tuned model

In [ ]:
# Load the ROUGE metric
rouge = evaluate.load("rouge")

# Extract the reference and prediction lists
references = data['Summaries'].tolist()      # Replace with your actual reference summary generated using llama for ground truth


predictions = data['bart_summary'].tolist()
predictions_finetuned = data['bartFinetuned_summary'].tolist()
# Compute ROUGE scores
result = rouge.compute(predictions=predictions, references=references, use_stemmer=True)
result_finetuned = rouge.compute(predictions=predictions_finetuned, references=references, use_stemmer=True)
print("ROUGE scores:", result)
print('-'*50)
print("ROUGE scores for Finetuned Bart large Xsum:", result_finetuned)

ROUGE scores: {'rouge1': np.float64(0.1959885059561169), 'rouge2': np.float64(0.0452511347242472), 'rougeL': np.float64(0.14621639074095727), 'rougeLsum': np.float64(0.1460426811907693)}
--------------------------------------------------
ROUGE scores for Finetuned Bart large Xsum: {'rouge1': np.float64(0.44209128282179355), 'rouge2': np.float64(0.19138509490388142), 'rougeL': np.float64(0.33595399171260054), 'rougeLsum': np.float64(0.335674328855457)}


##### METEOR score of pretrained and fine-tuned model

In [ ]:
# Load the METEOR metric
meteor = evaluate.load("meteor")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
import evaluate

# Extract the reference and prediction lists
references = data['Summaries'].tolist()      # Replace with your actual reference summary generated using llama for ground truth


predictions = data['bart_summary'].tolist()
predictions_finetuned = data['bartFinetuned_summary'].tolist()

# Compute the METEOR score
meteor_result = meteor.compute(predictions=predictions, references=references)
meteor_result_finetuned = meteor.compute(predictions=predictions_finetuned, references=references)
print("METEOR score for Bart large Xsum::", meteor_result)
print('-'*50)
print("METEOR score for Finetuned Bart large Xsum:", meteor_result_finetuned)


METEOR score for Bart large Xsum:: {'meteor': np.float64(0.1293776298273194)}
--------------------------------------------------
METEOR score for Finetuned Bart large Xsum: {'meteor': np.float64(0.38860390480688467)}


##### BERT score of pretrained and fine-tuned model

Run below cell if needed (if not executed earlier)

In [ ]:
!pip install bert_score

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 65.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 839.3 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 45.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
# Load the BERTScore metric
bertscore = evaluate.load("bertscore")

In [ ]:
# Extract the reference and prediction lists
references = data['Summaries'].tolist()      # Replace with your actual reference summary generated using llama for ground truth


predictions = data['bart_summary'].tolist()
predictions_finetuned = data['bartFinetuned_summary'].tolist()


# Compute BERTScore; you can set model_type, e.g., 'bert-base-uncased'
bertscore_result = bertscore.compute(predictions=predictions,
                                       references=references,
                                       model_type="bert-base-uncased")
bertscore_result_finetuned = bertscore.compute(predictions=predictions_finetuned,
                                       references=references,
                                       model_type="bert-base-uncased")
print("BERTScore for Bart large Xsum::", bertscore_result)
print('-'*50)
print("BERTScore for Finetuned Bart large Xsum:", bertscore_result_finetuned)

BERTScore for Bart large Xsum:: {'precision': [0.4941299557685852, 0.5094985365867615, 0.49588388204574585, 0.46193480491638184, 0.4491078555583954, 0.4416465163230896, 0.341824471950531, 0.6447777152061462, 0.43998128175735474, 0.40954193472862244, 0.5352227687835693, 0.40628308057785034, 0.47071897983551025, 0.5303062200546265, 0.5518076419830322, 0.3877105712890625, 0.47478410601615906, 0.5969356298446655, 0.4528518617153168, 0.55380779504776, 0.4780293107032776, 0.5962836742401123, 0.4884510636329651, 0.45817211270332336, 0.4870976209640503, 0.5074431896209717, 0.38943207263946533, 0.5435918569564819, 0.4973357617855072, 0.46695461869239807, 0.5783581733703613, 0.7129494547843933, 0.32323431968688965, 0.5673186779022217, 0.599840521812439, 0.5164193511009216, 0.5323963165283203, 0.46485698223114014, 0.5308587551116943, 0.5815725922584534, 0.6019875407218933, 0.4432358145713806, 0.4024812579154968, 0.45941364765167236, 0.3539533019065857, 0.5414334535598755, 0.4014090895652771, 0.59

In [ ]:
# Print overall scores
print('BERTScore analysis for BART Large Xsum on validation data')
print("Average Precision:", sum(bertscore_result['precision']) / len(bertscore_result['precision']))
print("Average Recall:", sum(bertscore_result['recall']) / len(bertscore_result['recall']))
print("Average F1:", sum(bertscore_result['f1']) / len(bertscore_result['f1']))

print('-'*50)
print('BERTScore analysis for Finetuned BART Large Xsum on validation data')
print("Average Precision:", sum(bertscore_result_finetuned['precision']) / len(bertscore_result_finetuned['precision']))
print("Average Recall:", sum(bertscore_result_finetuned['recall']) / len(bertscore_result_finetuned['recall']))
print("Average F1:", sum(bertscore_result_finetuned['f1']) / len(bertscore_result_finetuned['f1']))

BERTScore analysis for BART Large Xsum on validation data
Average Precision: 0.4773159898072481
Average Recall: 0.43636947486549615
Average F1: 0.4542268358916044
--------------------------------------------------
BERTScore analysis for Finetuned BART Large Xsum on validation data
Average Precision: 0.6454738554358482
Average Recall: 0.6639643733203411
Average F1: 0.6527462507784366


# Part 2: Chatbot

In [ ]:
!pip install transformers

## Upload file for fine-tuned model

Execute below cell if finetuned model is not already uploaded

In [ ]:
from google.colab import files
import zipfile
import os

# Upload the zip file
uploaded = files.upload()  # This will prompt you to upload the zip file manually

# Unzip the uploaded file
zip_filename = next(iter(uploaded))  # Gets the uploaded file name

# Create a directory (optional, if you want to extract to a specific folder)
extract_dir = './bartXsum_finetuned'
os.makedirs(extract_dir, exist_ok=True)

# Unzip
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print(f"Files extracted to: {extract_dir}")


Saving bartXsum_finetuned.zip to bartXsum_finetuned.zip
Files extracted to: ./bartXsum_finetuned


## Chatbot using finetuned model for summarization and text generation

In [ ]:
import torch
from transformers import BartForConditionalGeneration, BartTokenizer, AutoModelForCausalLM, AutoTokenizer

# Load the fine-tuned BART model for summarization
bart_model_path = "./bartXsum_finetuned"  # Path to your fine-tuned model
bart_model = BartForConditionalGeneration.from_pretrained(bart_model_path)
bart_tokenizer = BartTokenizer.from_pretrained("facebook/bart-large-xsum")

# Load the response generation model (DialoGPT, GPT-2, or similar)
response_model_name = "microsoft/DialoGPT-medium"  # You can change to GPT-2 or others
response_model = AutoModelForCausalLM.from_pretrained(response_model_name)
response_tokenizer = AutoTokenizer.from_pretrained(response_model_name)

# Store the conversation history
conversation_history = []

def summarize_history(history):
    """Summarize the entire conversation history."""
    dialogue = " ".join(history[-10:])  # Use the last 10 exchanges for summarization
    inputs = bart_tokenizer(dialogue, return_tensors="pt", max_length=1024, truncation=True)
    summary_ids = bart_model.generate(
        inputs["input_ids"],
        num_beams=4,
        min_length = 10,
        max_length=100,
        length_penalty=2.0)
    summary = bart_tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    print(f"Summarized Dialogue: {summary}")
    print("-" * 50)
    return summary


def generate_response(summary):
    """Generate a response using the summarized dialogue."""
    # Set the padding token to the EOS token if it's not defined

    #summary  = "Based on the following summary of our chat so far, continue the conversation as a friendly chatbot:\n" + summary
    if response_tokenizer.pad_token is None:
        response_tokenizer.pad_token = response_tokenizer.eos_token

    inputs = response_tokenizer(summary + response_tokenizer.eos_token, return_tensors="pt", padding=True, truncation=True)

    response_ids = response_model.generate(
        input_ids=inputs["input_ids"],
        attention_mask=inputs["attention_mask"],
        max_length=100,
        pad_token_id=response_tokenizer.eos_token_id,
        do_sample=True,
        top_k=70,
        top_p=0.95,
        temperature=0.6,
        repetition_penalty=1.2
    )

    # Get the input_ids tensor from the inputs dictionary
    input_ids_tensor = inputs["input_ids"]
    # Now use input_ids_tensor.shape
    response = response_tokenizer.decode(response_ids[:, input_ids_tensor.shape[-1]:][0], skip_special_tokens=True)
    return response


def chatbot():
    print("Chatbot: Hi! Let's chat. Type 'exit' to quit.")
    while True:
        user_input = input("You: ")

        if user_input.lower() == "exit":
            print("Chatbot: Goodbye!")
            break

        # Update conversation history
        conversation_history.append(user_input)

        # Summarize the dialogue history
        summary = summarize_history(conversation_history)

        # Generate a response based on the summarized dialogue
        response = generate_response(summary)

        # Print the chatbot's response
        print(f"Chatbot: {response}")

        # Append chatbot's response to history
        conversation_history.append(response)

if __name__ == "__main__":
    chatbot()


Chatbot: Hi! Let's chat. Type 'exit' to quit.
You: I will be going to my home town tomorrow
Summarized Dialogue: Person A is planning to travel to their home town for the weekend.
--------------------------------------------------
Chatbot: Well, I guess it's a good thing that she hasn't been there in years.
You: are you a dick?
Summarized Dialogue: Person A will be going to their home town tomorrow, which is a good thing since they haven't been there in years. Person A expresses regret about not seeing someone in years, calling the person a dick.
--------------------------------------------------
Chatbot: I don t think I ve ever seen someone like that.
You: Ok then please focus on what I'm telling to you, I will be going to my home tomorrow
Summarized Dialogue: Person A will be going to their home town tomorrow, which is a good thing since she hasn't been there in years. Person A expresses concern about someone being a "dick" and asks someone else to focus on what they're telling them.